# Radial basis functions

## Linear combination of general basis functions

This notebook contains studies on radial basis functions interpolation, to be used as part of surrogate optimization. This optimization method uses a *surrogate* (also known as metamodel, or surface response model) model to appoximate a given function $f$.

Follows the basic idea of approximating the function as a linear conbination of basis functions of $\vec{x}$:

\begin{equation}
\mathbf{h} = w_0\mathbf{\varphi_0}(\mathbf{x}) + w_1\mathbf{\varphi_1}(\mathbf{x})+\dots+w_n\mathbf{\varphi_n}(\mathbf{x})=\sum_{i=1}^n w_i\mathbf{\varphi_i}(\mathbf{x})
\end{equation}

Written in matrix notation:

\begin{equation}
    \mathbf{h} =
    \left[
    {\begin{array}{cc}
    \mathbf{\varphi_0}(x) & \mathbf{\varphi_1}(x) & \dots & \mathbf{\varphi_n}(x)
    \end{array} }
    \right]\mathbf{w}
\end{equation}

From this general form, derives linear, polynomial and other basis functions combinations.

## Radia basis functions

Radial basis functions interpolation uses a special set of functions, based on univariate (radial) functions and a set of observed data.

Given a set of $m$ points in $R^d$, let's refer to them as $\left\{\mathbf{x}_1,\ldots, \mathbf{x}_m \right\}$, where the function $f$ evaluation is known.